# Data preprocessing

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
import scanpy.external as sce
from scipy.sparse import csr_matrix
import os
import anndata
import warnings
import sys
# sys.path.append('/home/olga/prog/scoreCT/scorect/')
# import scorect_api as ct
warnings.filterwarnings('ignore')

In [3]:
path2tms = '../data/TMS/'

## Function: clean gene annotations

In [4]:
def clean_annotations(df):
    
    # Some gene names have flanking characters, remove them
    df.loc[:, 'Gene name'] = [str(g).strip('\t\n ') for g in df.loc[:, 'Gene name']]
    
    # More than one transcript per gene, get unique genes
    unique_genes = list(set(df.loc[:, 'Gene name']))
    
    # Remove nans if there are any (from df and from unique_genes)
    df = df[df.loc[:, 'Gene name'] != 'nan']
    unique_genes = [gene for gene in unique_genes if gene != 'nan']
    
    # Output dataframe: gene_annot
    gene_annot = pd.DataFrame(index=unique_genes)
    df.loc[:, 'gene_length'] = df.loc[:, 'Gene end (bp)'] - df.loc[:, 'Gene start (bp)']
    df.rename(columns={'Transcript length (including UTRs and CDS)': 'transcript_length'}, inplace=True)
    df = df.loc[:, ['Gene name', 'gene_length', 'transcript_length']]
    
    # Get min, max and mean transcript length associated with each gene
    min_length = df.loc[:, ['Gene name', 'transcript_length']].groupby(['Gene name']).min()
    max_length = df.loc[:, ['Gene name', 'transcript_length']].groupby(['Gene name']).max()
    mean_length = df.loc[:, ['Gene name', 'transcript_length']].groupby(['Gene name']).mean()
    
    # min_length.sort_index(inplace=True)
    
    gene_annot.loc[:, 'min_transcript_length'] = min_length
    gene_annot.loc[:, 'max_transcript_length'] = max_length
    gene_annot.loc[:, 'mean_transcript_length'] = mean_length
    
    # Add gene length annotations
    gene_annot.loc[:, 'gene_length'] = df.drop_duplicates('Gene name').set_index('Gene name').loc[gene_annot.index, 'gene_length']
    return gene_annot

In [5]:
genes_mouse = pd.read_csv('~/tmp/data/aging/mart_export_mouse.txt', sep='\t', index_col=0)
genes_human = pd.read_csv('~/tmp/data/aging/mart_export_human.txt', sep='\t', index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: '../../new_data/mart_export_mouse.txt'

In [18]:
genes_mouse

,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Chromosome/scaffold name,Strand,Gene end (bp),Gene start (bp),Transcript start (bp),Transcript end (bp),Transcription start site (TSS),Transcript length (including UTRs and CDS),Gene name,Gene % GC content
Gene stable ID,,,,,,,,,,,,,
ENSMUSG00000064336,ENSMUSG00000064336.1,ENSMUST00000082387,ENSMUST00000082387.1,MT,1,68,1,1,68,1,68,mt-Tf,30.88
ENSMUSG00000064337,ENSMUSG00000064337.1,ENSMUST00000082388,ENSMUST00000082388.1,MT,1,1024,70,70,1024,70,955,mt-Rnr1,35.81
ENSMUSG00000064338,ENSMUSG00000064338.1,ENSMUST00000082389,ENSMUST00000082389.1,MT,1,1093,1025,1025,1093,1025,69,mt-Tv,39.13
ENSMUSG00000064339,ENSMUSG00000064339.1,ENSMUST00000082390,ENSMUST00000082390.1,MT,1,2675,1094,1094,2675,1094,1582,mt-Rnr2,35.40
ENSMUSG00000064340,ENSMUSG00000064340.1,ENSMUST00000082391,ENSMUST00000082391.1,MT,1,2750,2676,2676,2750,2676,75,mt-Tl1,44.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000119806,ENSMUSG00000119806.1,ENSMUST00000240095,ENSMUST00000240095.1,2,1,151142999,151142864,151142864,151142999,151142864,136,Gm24077,44.12
ENSMUSG00000087609,ENSMUSG00000087609.2,ENSMUST00000129286,ENSMUST00000129286.2,2,1,151158167,151143894,151143894,151156640,151143894,542,4930442J19Rik,40.05
ENSMUSG00000087609,ENSMUSG00000087609.2,ENSMUST00000157058,ENSMUST00000157058.2,2,1,151158167,151143894,151143948,151158167,151143948,1721,4930442J19Rik,40.05


In [19]:
gene_annot_mouse = clean_annotations(genes_mouse)
gene_annot_human = clean_annotations(genes_human)

In [20]:
gene_annot_mouse.head(3)

,min_transcript_length,max_transcript_length,mean_transcript_length,gene_length
Mx1,352,2967,1796.3,15872
Gm3605,440,440,440.0,439
Gemin4,234,3540,1887.0,7093


In [21]:
gene_annot_human.head(3)

,min_transcript_length,max_transcript_length,mean_transcript_length,gene_length
KLF14,3511,3511,3511.000000,3510
RWDD2B,571,3065,1518.428571,14976
HMGN1P20,300,300,300.000000,299


## Check that we did not mess up

### Mouse

In [23]:
example1 = genes_mouse[genes_mouse.loc[:, 'Gene name'] == 'Il36rn']['Transcript length (including UTRs and CDS)']

In [24]:
end = genes_mouse[genes_mouse.loc[:, 'Gene name'] == 'Il36rn']['Gene end (bp)'][0]
start = genes_mouse[genes_mouse.loc[:, 'Gene name'] == 'Il36rn']['Gene start (bp)'][0]

In [25]:
print(f'Min: {example1.min()}\nMax: {example1.max()}\nMean: {example1.mean()}\nGene length: {end-start}')

Min: 347
Max: 2842
Mean: 1481.857142857143
Gene length: 6472


In [26]:
# Check that the values are the same
gene_annot_mouse.loc['Il36rn',:]

min_transcript_length      347.000000
max_transcript_length     2842.000000
mean_transcript_length    1481.857143
gene_length               6472.000000
Name: Il36rn, dtype: float64

### Human

In [27]:
example2 = genes_human[genes_human.loc[:, 'Gene name'] == 'HAX1']['Transcript length (including UTRs and CDS)']

In [28]:
end = genes_human[genes_human.loc[:, 'Gene name'] == 'HAX1']['Gene end (bp)'][0]
start = genes_human[genes_human.loc[:, 'Gene name'] == 'HAX1']['Gene start (bp)'][0]

In [29]:
print(f'Min: {example2.min()}\nMax: {example2.max()}\nMean: {example2.mean()}\nGene length: {end-start}')

Min: 464
Max: 1783
Mean: 1045.9583333333333
Gene length: 3520


In [30]:
# Check that the values are the same
gene_annot_human.loc['HAX1',:]

min_transcript_length      464.000000
max_transcript_length     1783.000000
mean_transcript_length    1045.958333
gene_length               3520.000000
Name: HAX1, dtype: float64

### Save mouse and human annotations

In [32]:
gene_annot_mouse.to_csv('../../new_data/mouse_annotations_clean.csv')
gene_annot_human.to_csv('../../new_data/human_annotations_clean.csv')

## Function: processing pipeline

In [ ]:
def processing(adata):
    """
    Run standard preprocessing pipeline: normalization, log-transformation, detection of HVGs, 
    Principal Component Analysis, batch-effect correction (per mouse), and UMAP. 
    """
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    sc.pp.pca(adata)
    sce.pp.bbknn(adata, batch_key='batch')
    sc.tl.umap(adata)
    
    return adata

## Function: add gene annotations to annData

In [ ]:
gene_annot_mouse.index

In [ ]:
def add_gene_annot_to_annData(adata, gene_annot):
    known_genes = list(set(adata.var_names).intersection(set(gene_annot.index)))
    for annot in ['gene_length', 'mean_transcript_length', 'min_transcript_length', 'max_transcript_length']: 
        adata.var.loc[known_genes, annot] = gene_annot.loc[known_genes, annot]
    return adata

## Function: harmonize TMS annotations
* from ```cell_ontology_class``` to ```cell type```
* from ```mouse.id``` to ```batch```
* from ```age``` to ```age_number```
* create category ```age``` that contains ```young``` and ```old``` annotations


In [ ]:
def harmonize_TMS_annotations(adata):
    age_dic = {'1m': 'young', '3m': 'young', '18m': 'old', '21m': 'old', '24m': 'old', '30m': 'old'}
    
    adata.obs.rename(columns = {'cell_ontology_class':'cell_type', 'mouse.id': 'batch'}, inplace=True)
    adata.obs['age_number'] = [age_dic[age] for age in adata.obs['age']]
    adata.obs['age'] = [age_dic[age] for age in adata.obs['age']]
    # remove empty and unnecessary annotations
    adata.obs.drop(['cell_ontology_id','method', 'free_annotation', 'louvain', 'leiden'], 
                   axis=1, inplace=True)
    adata.obs['age'] = adata.obs['age'].astype('category')
    adata.obs['age'].cat.reorder_categories(['young', 'old'], inplace=True) # so that the young are always plotted first

## Function: plot UMAPS

In [ ]:
bold_and_vivid = ['#7F3C8D','#11A579','#3969AC','#F2B701','#E73F74','#80BA5A','#E68310','#008695','#CF1C90',
           '#f97b72','#4b4b8f', '#E58606','#5D69B1','#52BCA3','#99C945','#CC61B0','#24796C','#DAA51B',
           '#2F8AC4','#764E9F','#ED645A','#CC3A8E']
safe = ['#88CCEE', '#CC6677', '#DDCC77', '#117733', '#332288', '#AA4499', 
        '#44AA99', '#999933', '#882255', '#661100', '#6699CC']

age_palette = ['#557e96ff', '#7C3E66']


def plot_UMAPs(adata, dset): 
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(8, 3))
    sc.pl.umap(adata, ax=ax0, color='age', palette=age_palette, show=False, s=40)
    sc.pl.umap(adata, ax=ax1,color='batch', palette=safe+bold_and_vivid, show=False, s=40)
    ax0.set_xlabel('')
    ax1.set_xlabel('')
    ax0.set_ylabel('')
    ax1.set_ylabel('')
    plt.tight_layout()
    plt.savefig(f'../output/figs/balanced/UMAPs_age_batch/png/{dset}.png', dpi=300)
    plt.savefig(f'../output/figs/balanced/UMAPs_age_batch/svg/{dset}.svg')

## Function: whole pipeline
* Harmonization of age and batch annotations
* Standard processing: log, DR, batch-effect correction
* Add gene annotations (gene and min/max/mean transcript length)
* Generate UMAPs and save them in /balanced/UMAPS_age_batch/
* Save annDAta in annDatas_balanced/

In [ ]:
def TMS_pipeline(adata, dset, dset_type): 
    harmonize_TMS_annotations(adata)
    processing(adata)
    add_gene_annot_to_annData(adata, gene_annot_mouse)
    plot_UMAPs(adata, f'{dset}_{dset_type}')
    adata.write(f'../data/annDatas_balanced/{dset}_{dset_type}.h5ad')

# Liver

##  Liver (TMS FACS)
We keep male mice: 
* 2 mice aged 3 months
* 2 mice aged 24 months. 

In [ ]:
adata = sc.read(path2tms + 'tabula-muris-senis-facs-processed-official-annotations-Liver.h5ad')
dset = 'TMS_FACS_liver'

In [ ]:
sc.pl.umap(adata, color=['mouse.id', 'subtissue'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]

In [ ]:
sc.pl.umap(adata_male_3_24, color='mouse.id')

In [ ]:
for dset_type in ['male_3_24']: 
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

# Muscle

## Limb muscle (TMS FACS)
We create two datasets.

**Male dataset**: 
* 4 male mice aged 3 months
* 4 male mice aged 24 months. 

**Female dataset**: 
* 2 female mice aged 3 months
* 2 female mice aged 24 months. 

In [ ]:
adata = sc.read(path2tms+'/tabula-muris-senis-facs-processed-official-annotations-Limb_Muscle.h5ad')
dset = 'TMS_FACS_muscle'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
for dset_type in ['male_3_24', 'female_3_18']: 
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

In [ ]:
# Save whole dataset:
adata = sc.read(path2tms+'/tabula-muris-senis-facs-processed-official-annotations-Limb_Muscle.h5ad')
dset = 'TMS_FACS_muscle'
TMS_pipeline(adata, dset, 'all_mice')

## Limb muscle (TMS droplet)

In [ ]:
adata = sc.read(path2tms+'/tabula-muris-senis-droplet-processed-official-annotations-Limb_Muscle.h5ad')
dset = 'TMS_droplet_muscle'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_1_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['1m', '24m']))]
adata_female_3_21 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '21m']))]

In [ ]:
sc.pl.umap(adata_male_1_24, color='age')

In [ ]:
sc.pl.umap(adata_female_3_21, color='age')

In [ ]:
for dset_type in ['male_1_24', 'female_3_21']: 
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

In [ ]:
# Save whole dataset:
adata = sc.read(path2tms+'/tabula-muris-senis-droplet-processed-official-annotations-Limb_Muscle.h5ad')
dset = 'TMS_droplet_muscle'
TMS_pipeline(adata, dset, 'all_mice')

# Brain 

##  Brain non-myeloid (TMS FACS)

**Male dataset**: 
* 4 male mice aged 3 months
* 4 male mice aged 24 months

**Female dataset**:
* 3 female mice aged 3 months
* 2 female mice aged 18 months

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-facs-processed-official-annotations-Brain_Non-Myeloid.h5ad')
dset = 'TMS_FACS_brain_non_myeloid'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
for dset_type in ['male_3_24', 'female_3_18']: 
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

## Brain myeloid (TMS FACS)  (NOT DONE)

In [ ]:
adata = sc.read(path2tms + 'tabula-muris-senis-facs-processed-official-annotations-Brain_Myeloid.h5ad')
dset = 'TMS_FACS_brain_myeloid'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_18 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '18m']))]
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
for dset_type in ['male_3_18', 'male_3_24', 'female_3_18']: 
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

## Brain (Ximerakis)

In [ ]:
adata = sc.read('../data/annDatas_processed/Ximerakis.h5ad')

In [ ]:
adata.X = csr_matrix(adata.X)

In [ ]:
sc.pl.umap(adata, color='age')

In [ ]:
adata = add_gene_annot_to_annData(adata, gene_annot_mouse)

In [ ]:
age_dic = {'old': '22m', 'young': '2.5m'}
adata.obs['age_number'] = [age_dic[age] for age in adata.obs['age']]

In [ ]:
adata.write('../data/annDatas_balanced/Ximerakis_male_2_22.h5ad')

# Heart

## Heart and aorta (TMS droplet) 


**Male dataset 1_18**
* 1 male mouse aged 1 month
* 2 male mice aged 18 months

**Male dataset 1_24**:
* 1 male mouse aged 1 month
* 2 male mice aged 24 months

**Female dataset 3_18**:
* 1 female mouse aged 3 months
* 1 female mouse aged 18 months

**Female dataset 3_21**:
* 1 female mouse aged 3 months
* 2 female mice aged 21 months

In [ ]:
adata = sc.read(path2tms + 'tabula-muris-senis-droplet-processed-official-annotations-Heart_and_Aorta.h5ad')
dset = 'TMS_droplet_heart_and_aorta'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_1_18 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['1m', '18m']))]
adata_male_1_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['1m', '24m']))]

adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]
adata_female_3_21 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '21m']))]

In [ ]:
for dset_type in ['male_1_18', 'male_1_24', 'female_3_18', 'female_3_21']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

In [ ]:
# Save whole dataset:
adata = sc.read(path2tms + 'tabula-muris-senis-droplet-processed-official-annotations-Heart_and_Aorta.h5ad')
dset = 'TMS_droplet_heart'
TMS_pipeline(adata, dset, 'all_mice')

## Heart (TMS FACS)

In [ ]:
adata = sc.read(path2tms + 'tabula-muris-senis-facs-processed-official-annotations-Heart.h5ad')
dset = 'TMS_FACS_heart'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
sc.pl.umap(adata_male_3_24, color='age')

In [ ]:
sc.pl.umap(adata_female_3_18, color='age')

In [ ]:
for dset_type in ['male_3_24', 'female_3_18']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

In [ ]:
# Save whole dataset: 
adata = sc.read(path2tms + 'tabula-muris-senis-facs-processed-official-annotations-Heart.h5ad')
dset = 'TMS_FACS_heart'
TMS_pipeline(adata, dset, 'all_mice')

# Thymus

## Thymus (TMS FACS) 

**Male dataset**
* 3 male mice aged 3 months
* 4 male mice aged 24 months

**Female dataset**
* 2 female mice aged 3 months
* 2 female mice aged 18 months

In [ ]:
adata = sc.read(path2tms + 'tabula-muris-senis-facs-processed-official-annotations-Thymus.h5ad')
dset = 'TMS_FACS_thymus'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
for dset_type in ['male_3_24', 'female_3_18']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

# Skin

## Skin (TMS FACS)

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-facs-processed-official-annotations-Skin.h5ad')
dset = 'TMS_FACS_skin'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
for dset_type in ['male_3_24', 'female_3_18']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

## Skin (TMS droplet) 
There are no young individuals. 

In [ ]:
adata = sc.read(path2tms + 'tabula-muris-senis-droplet-processed-official-annotations-Skin.h5ad')
dset = 'TMS_droplet_skin'

In [ ]:
sc.pl.umap(adata, color='mouse.id')

# Bladder

## Bladder (TMS droplet)

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-droplet-processed-official-annotations-Bladder.h5ad')
dset = 'TMS_droplet_bladder'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
TMS_pipeline(adata_male_3_24, dset, 'male_3_24')

# Kidney

## TMS droplet kidney

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-droplet-processed-official-annotations-Kidney.h5ad')
dset = 'TMS_droplet_kidney'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
sc.pl.umap(adata[adata.obs['age'].isin(['1m', '3m', '30m'])], color=['mouse.id'])

There are no young and old samples from the same subtissue of same sex individuals.

## TMS FACS kidney

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-facs-processed-official-annotations-Kidney.h5ad')
dset = 'TMS_FACS_kidney'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_18 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '18m']))]
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
for dset_type in ['male_3_18', 'male_3_24', 'female_3_18']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

## Kimmel: kidney
The processing of all three Kimmel datasets is below, in the "Kimmel" section. 

# Lung

## Lung (TMS FACS)

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-facs-processed-official-annotations-Lung.h5ad')
dset= 'TMS_FACS_lung'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
sc.pl.umap(adata_female_3_18, color=['mouse.id', 'age'])

In [ ]:
for dset_type in ['male_3_24', 'female_3_18']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

## Lung (Angelidis)

In [ ]:
adata = sc.read('../data/annDatas_processed/Angelidis_annotated.h5ad')
dset = 'Angelidis'

In [ ]:
adata.obs['batch'] = adata.obs['mouse']

In [ ]:
sc.pl.umap(adata, color=['mouse', 'age'])

In [ ]:
plot_UMAPs(adata, 'Angelidis')

In [ ]:
# Simplify cell type annotation
sc.pl.umap(adata, color='cell_type')

In [ ]:
set(adata.obs['cell_type'])

In [ ]:
cell_type_dic = {
    'Alveolar macrophage': 'Alveolar macrophage',
 'B cells': 'B cell',
 'Capillary endothelial cells': 'Endothelial', 
 'Ccl17+/cd103-/cd11b- dendritic cells': 'Dendritic cell',
 'Cd103+/cd11b- dendritic cells': 'Dendritic cell', 
 'Cd209+/cd11b+ dendritic cells': 'Dendritic cell', 
 'Cd4+ t cells': 'T cell', 
 'Cd8+ t cells': 'T cell', 
 'Ciliated cells': 'Ciliated', 
 'Classical monocyte (ly6c2+)': 'Monocyte', 
 'Club cells': 'Club cell', 
 'Eosinophils': 'Granulocyte', 
 'Fn1+ macrophage': 'Macrophage', 
 'Gamma-delta t cells': 'T cell', 
 'Goblet cells': 'Goblet cell', 
 'Interstitial fibroblast': 'Fibroblast', 
 'Interstitial macrophages': 'Macrophage', 
 'Lipofibroblast': 'Fibroblast', 
 'Lymphatic endothelial cells': 'Endothelial', 
 'Megakaryocytes': 'Megakaryocyte', 
 'Mesothelial cells': 'Mesothelial', 
 'Mki67+ proliferating cells': 'Profilerating', 
 'Natural killer cells': 'NK cell', 
 'Neutrophils': 'Granulocyte', 
 'Non-classical monocyte (ly6c2-)': 'Monocyte', 
 'Plasma cells': 'Plasma cell', 
 'Red blood cells': 'Erythrocyte', 
 'Smooth muscle cells': 'Smooth muscle', 
 'Type 1 pneumocytes': 'Pneumocyte', 
 'Type 2 pneumocytes': 'Pneumocyte', 
 'Vascular endothelial cells': 'Endothelial', 
 'Vcam1+ endothelial cells': 'Endothelial'}

In [ ]:
adata.obs['cell_type_original'] = adata.obs['cell_type']
adata.obs['cell_type'] = [cell_type_dic[ct] for ct in adata.obs['cell_type']]

In [ ]:
sc.pl.umap(adata, color=['cell_type_original', 'cell_type'], ncols=1)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='cell_type')

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).to_csv('../output/tables/Angelidis_ct_markers.csv')

In [ ]:
adata = add_gene_annot_to_annData(adata, gene_annot_mouse)

In [ ]:
adata.write('../data/annDatas_balanced/Angelidis_lung_3_24.h5ad')

# Pancreas

## Pancreas (TMS FACS) 

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-facs-processed-official-annotations-Pancreas.h5ad')
dset = 'TMS_FACS_pancreas'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
for dset_type in ['male_3_24', 'female_3_18']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

## Pancreas (TMS droplet) 
There are no young individuals. 

# Mammary gland

## Mammary gland (TMS droplet)

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-droplet-processed-official-annotations-Mammary_Gland.h5ad')
dset = 'TMS_droplet_mammary_gland'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_female_3_18 = adata[adata.obs['age'].isin(['3m', '18m'])]
adata_female_3_21 = adata[adata.obs['age'].isin(['3m', '21m'])]

In [ ]:
for dset_type in ['female_3_18', 'female_3_21']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

## Mammary gland (TMS FACS) (NOT DONE)

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-facs-processed-official-annotations-Mammary_Gland.h5ad')
dset = 'TMS_FACS_mammary_gland'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_female_3_18 = adata[adata.obs['age'].isin(['3m', '18m'])]
adata_female_3_21 = adata[adata.obs['age'].isin(['3m', '21m'])]

In [ ]:
for dset_type in ['female_3_18', 'female_3_21']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

# Spleen

## Spleen (TMS FACS) 

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-facs-processed-official-annotations-Spleen.h5ad')
dset = 'TMS_FACS_spleen'

In [ ]:
sc.pl.umap(adata, color=['mouse.id'])

In [ ]:
adata_male_3_24 = adata[(adata.obs['sex'] == 'male') & (adata.obs['age'].isin(['3m', '24m']))]
adata_female_3_18 = adata[(adata.obs['sex'] == 'female') & (adata.obs['age'].isin(['3m', '18m']))]

In [ ]:
for dset_type in ['male_3_24', 'female_3_18']:
    adata = eval(f'adata_{dset_type}')
    TMS_pipeline(adata, dset, dset_type)

In [ ]:
adata = sc.read(path2tms+'tabula-muris-senis-facs-processed-official-annotations-Spleen.h5ad')
sc.pl.umap(adata, color='cell_ontology_class')

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='cell_ontology_class')

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).to_csv('../output/tables/TMS_spleen_ct_markers.csv')

## Kimmel: spleen
Processing inside the "Kimmel" section. 

# Raredon

In [ ]:
adata = sc.read('../data/annDatas_processed/Raredon_young_old.h5ad')

In [ ]:
adata.X = csr_matrix(adata.X)

In [ ]:
adata.obs['age'] = [age.lower() for age in adata.obs['age']]

In [ ]:
adata = processing(adata)

In [ ]:
adata = add_gene_annot_to_annData(adata, gene_annot_human)

In [ ]:
adata.obs['age_number'] = adata.obs['age_number'].astype('category')

In [ ]:
sc.pl.umap(adata, color='age_number')

In [ ]:
adata.obs['batch']

In [ ]:
sc.pl.umap(adata, color=['age', 'cell_type'])

In [ ]:
adata.write('../data/annDatas_balanced/Raredon_lung_mixed_27_73.h5ad')

# Travaglini (human lung)

Better to download the whole dataset again and take all cells instead of just 7'500 per age cohort. Or at least the same cell types. 

In [ ]:
adata = sc.read('../data/HLCA/Travaglini_young_old.h5ad')

In [ ]:
adata.X = csr_matrix(adata.X)

In [ ]:
adata.obs['age_number'] = adata.obs['age']

In [ ]:
age_dic =  {46: 'young', 75:'old'}
adata.obs['age'] = [age_dic[age] for age in adata.obs['age_number']]

In [ ]:
adata.obs['batch'] = adata.obs['patient']

In [ ]:
adata = add_gene_annot_to_annData(adata, gene_annot_human)

In [ ]:
adata.write('../data/annDatas_balanced/Travaglini_lung_male_46_75.h5ad')

# Solé-Boldó (human skin)

In [ ]:
adata = sc.read('../data/annDatas_processed/Sole_Boldo.h5ad')

In [ ]:
adata.X = csr_matrix(adata.X)

In [ ]:
adata.obs['age'] = [age.lower() for age in adata.obs['condition']]

In [ ]:
adata = add_gene_annot_to_annData(adata, gene_annot_human)

In [ ]:
adata.write('../data/annDatas_balanced/SoleBoldo_skin_male_26_62.h5ad')

# Enge (human pancreas)

In [ ]:
adata = sc.read('../data/annDatas_processed/Enge_annotated.h5ad')
adata.X = csr_matrix(adata.X)

In [ ]:
adata = adata[adata.obs['age'].isin([21, 22, 44, 54])]

In [ ]:
sc.pl.umap(adata, color=['sex', 'donor'])

In [ ]:
adata.obs['batch'] = adata.obs['donor']

In [ ]:
age_dic = {21: 'young', 22: 'young', 44: 'old', 54:'old'}
adata.obs['age_number'] = adata.obs['age']
adata.obs['age'] = [age_dic[age] for age in adata.obs['age_number']]

In [ ]:
adata = add_gene_annot_to_annData(adata, gene_annot_human)

In [ ]:
adata.write('../data/annDatas_balanced/Enge_pancreas_mixed_22_49.h5ad')

# Salzer (murine dermal fibroblasts)

In [ ]:
adata = sc.read('../data/annDatas_processed/Salzer_preprocessed.h5ad')
adata.X = csr_matrix(adata.X)

In [ ]:
adata = adata[adata.obs['age'] != 'newborn']

In [ ]:
adata = add_gene_annot_to_annData(adata, gene_annot_mouse)

In [ ]:
adata.write('../data/annDatas_balanced/Salzer_fibroblasts_female_2_18.h5ad')

# Kimmel (all tissues)

In [ ]:
path2input = '../data/Kimmel/gsm/'

tissue_dic = {'L': 'lung', 'K': 'kidney', 'S': 'spleen'}
age_dic = {'O': 'old', 'Y': 'young'}

fnames = os.listdir('../data/Kimmel/gsm')
adata_list = []
for fname in fnames:
    adata_x = sc.read_10x_h5(f'{path2input}{fname}')
    adata_x.var_names_make_unique()
    adata_x.obs['tissue'] = tissue_dic[fname.split('_')[1][2]]
    adata_x.obs['age'] = age_dic[fname.split('_')[1][0]]
    adata_x.obs['sample'] = fname.split('_')[1]
    adata_x.obs['batch'] = fname.split('_')[1][:2]
    adata_list.append(adata_x)

In [ ]:
adata = anndata.AnnData.concatenate(*adata_list)

In [ ]:
adata_lung = adata[adata.obs['tissue']=='lung']
adata_kidney = adata[adata.obs['tissue']=='kidney']
adata_spleen = adata[adata.obs['tissue'] == 'spleen']

## Kimmel lung

In [ ]:
adata_lung = processing(adata_lung)

In [ ]:
adata_lung = add_gene_annot_to_annData(adata_lung, gene_annot_mouse)

In [ ]:
adata_lung.write('../data/annDatas_balanced/Kimmel_lung_male_7_21.h5ad')

In [ ]:
sc.tl.leiden(adata_lung)

In [ ]:
sc.pl.umap(adata_lung, color='leiden')

In [ ]:
# ref_marker = ct.read_markers_from_file('/home/amon/scoreCT/data/ref_marker.gmt')
ref_marker = pd.read_csv('../output/tables/Angelidis_ct_markers.csv', index_col=0).iloc[0:300, :]

In [ ]:
marker_df = ct.wrangle_ranks_from_anndata(adata_lung)

In [ ]:
# Score cell types for each cluster 
# Let's set parameters first - K represents the number of genes included in the ranking
# m represents the number of bins used to divide the top K genes.
K = 300
m = 5
# Get the background genes - here, all the genes used to run the differential gene expression test
background = adata.var_names.tolist()
# Now run the function
ct_pval, ct_score = ct.celltype_scores(nb_bins=m,
                                        ranked_genes=marker_df,
                                        K_top = K,
                                        marker_ref=ref_marker,
                                        background_genes=background)


In [ ]:
# ct.plot_pvalue(ct_pval, clusters=['0','1'], n_types=3)

In [ ]:
# Now assign clusters to cell types
cluster_assign = adata.obs['leiden']
celltype_assign = ct.assign_celltypes(cluster_assignment=cluster_assign, ct_pval_df=ct_pval, ct_score_df=ct_score, cutoff=0.1)
# Add to anndata object
adata.obs['cell_type'] = celltype_assign
# Let's compare with the true assignment now! 
sc.pl.umap(adata, color=['leiden','cell_type'], title=['True','Predicted'], cmap='Set2')

### Cell type annotation

In [ ]:
adata_lung.obs

In [ ]:
sc.pl.umap(adata_lung, color='leiden')

In [ ]:
sc.tl.rank_genes_groups(adata_lung, groupby='leiden')

In [ ]:
ref_marker = pd.read_csv('../output/tables/Angelidis_ct_markers.csv', index_col=0).iloc[0:300, :]
marker_df = ct.wrangle_ranks_from_anndata(adata_lung)

In [ ]:
# Score cell types for each cluster 
# Let's set parameters first - K represents the number of genes included in the ranking
# m represents the number of bins used to divide the top K genes.
K = 300
m = 5
# Get the background genes - here, all the genes used to run the differential gene expression test
background = adata.var_names.tolist()
# Now run the function
ct_pval, ct_score = ct.celltype_scores(nb_bins=m,
                                        ranked_genes=marker_df,
                                        K_top = K,
                                        marker_ref=ref_marker,
                                        background_genes=background)

In [ ]:
# Now assign clusters to cell types
cluster_assign = adata_lung.obs['leiden']
celltype_assign = ct.assign_celltypes(cluster_assignment=cluster_assign, ct_pval_df=ct_pval, ct_score_df=ct_score, cutoff=0.1)
# Add to anndata object
adata_lung.obs['cell_type'] = celltype_assign
# Let's compare with the true assignment now! 
sc.pl.umap(adata_lung, color=['leiden','cell_type'], title=['True','Predicted'], cmap='Set2')

In [ ]:
adata_lung = add_gene_annot_to_annData(adata_lung, gene_annot_mouse)

In [ ]:
adata_lung

## Kimmel spleen

In [ ]:
adata_spleen = processing(adata_spleen)

In [ ]:
adata_spleen = add_gene_annot_to_annData(adata_spleen, gene_annot_mouse)

In [ ]:
adata_spleen.write('../data/annDatas_balanced/Kimmel_spleen_male_7_21.h5ad')

In [ ]:
def scoreCT_pipeline(adata,ref_marker):
    marker_df = ct.wrangle_ranks_from_anndata(adata)
    # Score cell types for each cluster 
    # Let's set parameters first - K represents the number of genes included in the ranking
    # m represents the number of bins used to divide the top K genes.
    K = 300
    m = 5
    # Get the background genes - here, all the genes used to run the differential gene expression test
    background = adata.var_names.tolist()
    # Now run the function
    ct_pval, ct_score = ct.celltype_scores(nb_bins=m,
                                            ranked_genes=marker_df,
                                            K_top = K,
                                            marker_ref=ref_marker,
                                            background_genes=background)
    # Now assign clusters to cell types
    cluster_assign = adata.obs['leiden']
    celltype_assign = ct.assign_celltypes(cluster_assignment=cluster_assign, ct_pval_df=ct_pval, ct_score_df=ct_score, cutoff=0.1)
    # Add to anndata object
    adata.obs['cell_type'] = celltype_assign

In [ ]:
adata_spleen = processing(adata_spleen)

## Cell type annotation

In [ ]:
# sc.tl.leiden(adata_spleen)

In [ ]:
# sc.tl.rank_genes_groups(adata_spleen, groupby='leiden')

In [ ]:
# ref_marker = pd.read_csv('../output/tables/TMS_spleen_ct_markers.csv', index_col=0).iloc[0:300, :]
# marker_df = ct.wrangle_ranks_from_anndata(adata_spleen)

In [ ]:
# scoreCT_pipeline(adata_spleen, ref_marker)

In [ ]:
# adata_spleen = add_gene_annot_to_annData(adata_spleen, gene_annot_mouse)

In [ ]:
# sc.pl.umap(adata_spleen, color='cell_type')

In [ ]:
# sc.pl.umap(adata_spleen, color='leiden')

In [ ]:
# adata_spleen.write('../data/annDatas_balanced/Kimmel_spleen.h5ad')

## Kimmel kidney

In [ ]:
adata_kidney = processing(adata_kidney)

In [ ]:
adata_kidney = add_gene_annot_to_annData(adata_kidney, gene_annot_mouse)

In [ ]:
adata_kidney.write('../data/annDatas_balanced/Kimmel_kidney_male_7_21.h5ad')

# UV-radiated skin

In [ ]:
adata = sc.read('../data/annDatas_processed/UV_radiated_skin.h5ad')

In [ ]:
adata = add_gene_annot_to_annData(adata, gene_annot_mouse)

In [ ]:
adata.write('../data/annDatas_processed/UV_radiated_skin.h5ad')

# Human trachea of smokers (Goldfarbmuren)

In [ ]:
adata = sc.read('../data/annDatas_processed/Goldfarbmuren_annotated.h5ad')

In [ ]:
adata = add_gene_annot_to_annData(adata, gene_annot_human)

In [ ]:
adata.write('../data/annDatas_processed/Goldfarbmuren_annotated_with_gene_length.h5ad')

# Global aging genes

In [ ]:
gags = pd.read_csv('../data/TMS/global_aging_genes.tsv', sep='\t', index_col=0)

In [ ]:
gene_annot_mouse = pd.read_csv('../data/biomart/mouse_annotations_clean.csv', index_col=0)

In [ ]:
gags.loc[:, 'gene_length'] = [gene_annot_mouse.loc[gene,'gene_length'] if gene in gene_annot_mouse.index else np.nan for gene in gags.index]

In [ ]:
up_idx = gags[gags.prop_of_up_regulated_tissue_cell_types > 0.5].index
down_idx = gags[gags.prop_of_up_regulated_tissue_cell_types < 0.5].index

In [ ]:
gags.loc[up_idx, 'up/down'] = 'up'
gags.loc[down_idx, 'up/down'] = 'down'

In [ ]:
gags.to_csv('../output/tables/global_aging_genes.csv')